In [1]:
import pandas as pd 
import random
from datetime import datetime, timedelta

In [2]:
# Function to generate random latitude and longitude for each parking lot
def generate_random_location():
    latitude = random.uniform(-90, 90)  # Random latitude between -90 and 90
    longitude = random.uniform(-180, 180)  # Random longitude between -180 and 180
    return latitude, longitude

In [4]:
#CSV format 
df = pd.read_csv("BaySensors.csv")

FileNotFoundError: [Errno 2] No such file or directory: 'BaySensors.csv'

In [3]:
df.rename(columns={'Zone_Number': 'Parking Lot'}, inplace=True)
df = df.drop(['Lastupdated', 'KerbsideID'], axis=1)
df = df.dropna(subset=['Parking Lot'])

# Split the 'Location' column into two separate columns: 'Latitude' and 'Longitude'
df[['Latitude', 'Longitude']] = df['Location'].str.split(',', expand=True)
df = df.drop(columns=['Location'])

df['Parking Lot'].isna().sum()

# Conversion
df['Latitude'] = df['Latitude'].astype(float)
df['Longitude'] = df['Longitude'].astype(float)
df['Parking Lot'] = df['Parking Lot'].astype(int)

df['Latitude'] = df['Latitude'].round(3)
df['Longitude'] = df['Longitude'].round(3)



NameError: name 'df' is not defined

In [5]:
# Remove the part after '+' in 'Status_Timestamp'
df['Status_Timestamp'] = df['Status_Timestamp'].str.split('+').str[0]
# Step 2: Split 'Status_Timestamp' into 'Date' and 'Time' by separating at 'T'
df[['Date', 'Time']] = df['Status_Timestamp'].str.split('T', expand=True)

df = df.drop(columns=['Status_Timestamp'])
df

,Parking Lot,Status_Description,Latitude,Longitude,Date,Time
0,7084,Present,-37.802,144.961,2024-11-28,04:03:02
1,7084,Present,-37.802,144.961,2024-11-28,03:31:34
2,7084,Present,-37.802,144.961,2024-11-28,02:08:40
3,7084,Unoccupied,-37.802,144.961,2024-11-28,03:53:41
4,7084,Unoccupied,-37.802,144.961,2024-11-28,03:57:26
...,...,...,...,...,...,...
3302,7389,Unoccupied,-37.813,144.963,2024-12-29,13:42:53
3303,7389,Present,-37.813,144.963,2025-01-16,08:20:22
3304,7096,Present,-37.807,144.965,2025-01-16,12:07:29
3305,7770,Present,-37.812,144.964,2024-10-21,03:35:45


In [6]:
def modify_and_append_rows(df):
    # List to store the new rows
    new_rows = []
    
    for index, row in df.iterrows():
        # Copy the row to avoid modifying the original
        new_row = row.copy()
        
        # Randomly change the Status_Description between 'Present' and 'Unoccupied'
        new_row['Status_Description'] = random.choice(['Present', 'Unoccupied'])
        
        # Combine 'Date' and 'Time' columns into a single datetime object
        datetime_str = new_row['Date'] + ' ' + new_row['Time']
        datetime_obj = datetime.strptime(datetime_str, '%Y-%m-%d %H:%M:%S')
        
        # Increment the time by 1 minute
        new_datetime_obj = datetime_obj + timedelta(minutes=1)
        
        # Update the 'Time' column with the new time
        new_row['Time'] = new_datetime_obj.strftime('%H:%M:%S')
        
        # Append the modified row to the list of new rows
        new_rows.append(new_row)
    
    # Convert the list of new rows back into a DataFrame
    new_df = pd.DataFrame(new_rows)
    
    # Optionally, you can append this new DataFrame to the original one or return it
    return pd.concat([df, new_df], ignore_index=True)



df = pd.DataFrame(df)
df = modify_and_append_rows(df)
df = df.sort_values(by=['Parking Lot', 'Time'])


In [7]:
df

,Parking Lot,Status_Description,Latitude,Longitude,Date,Time
235,7010,Present,-37.805,144.960,2024-10-11,00:38:32
3325,7010,Present,-37.805,144.960,2024-10-11,00:39:32
234,7010,Present,-37.805,144.960,2024-10-11,00:53:01
3324,7010,Unoccupied,-37.805,144.960,2024-10-11,00:54:01
1208,7010,Present,-37.805,144.960,2024-10-11,01:16:06
...,...,...,...,...,...,...
3396,7993,Unoccupied,-37.807,144.954,2025-01-15,16:42:21
2325,7993,Unoccupied,-37.807,144.954,2025-01-15,22:21:07
5415,7993,Unoccupied,-37.807,144.954,2025-01-15,22:22:07
1279,7995,Unoccupied,-37.808,144.954,2025-01-16,11:48:13
